In [85]:
#importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection, preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [6]:
df=pd.read_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\train_Df64byy.csv')

In [7]:
df1=df.copy()

In [4]:
df.isnull().sum()

ID                             0
City_Code                      0
Region_Code                    0
Accomodation_Type              0
Reco_Insurance_Type            0
Upper_Age                      0
Lower_Age                      0
Is_Spouse                      0
Health Indicator           11691
Holding_Policy_Duration    20251
Holding_Policy_Type        20251
Reco_Policy_Cat                0
Reco_Policy_Premium            0
Response                       0
dtype: int64

In [5]:
df.T.apply(lambda x: x.nunique(), axis=1)

ID                         50882
City_Code                     36
Region_Code                 5316
Accomodation_Type              2
Reco_Insurance_Type            2
Upper_Age                     58
Lower_Age                     60
Is_Spouse                      2
Health Indicator               9
Holding_Policy_Duration       15
Holding_Policy_Type            4
Reco_Policy_Cat               22
Reco_Policy_Premium         6977
Response                       2
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 14 columns):
ID                         50882 non-null int64
City_Code                  50882 non-null object
Region_Code                50882 non-null int64
Accomodation_Type          50882 non-null object
Reco_Insurance_Type        50882 non-null object
Upper_Age                  50882 non-null int64
Lower_Age                  50882 non-null int64
Is_Spouse                  50882 non-null object
Health Indicator           39191 non-null object
Holding_Policy_Duration    30631 non-null object
Holding_Policy_Type        30631 non-null float64
Reco_Policy_Cat            50882 non-null int64
Reco_Policy_Premium        50882 non-null float64
Response                   50882 non-null int64
dtypes: float64(2), int64(6), object(6)
memory usage: 5.4+ MB


In [8]:
df['Holding_Policy_Duration']=df['Holding_Policy_Duration'].replace('14+',15)

In [9]:
df=df.astype({"Holding_Policy_Type":'category', "Reco_Policy_Cat":'category',"City_Code":'category',"Accomodation_Type":'category',"Reco_Insurance_Type":'category',"Holding_Policy_Duration":'category',"Is_Spouse":'category',"Health Indicator":'category','Region_Code':'category'})

In [10]:
#No missing values in num variables
num_vars=df.columns[df.dtypes!='category']
cat_vars=df.columns[df.dtypes=='category']

# MISSING VALUE ANALYSIS & IMPUTATION

In [11]:
df['Health Indicator']=df['Health Indicator'].cat.codes
df['Health Indicator']=df['Health Indicator'].replace(-1, np.nan)
df['Accomodation_Type']=df['Accomodation_Type'].cat.codes
df['Reco_Insurance_Type']=df['Reco_Insurance_Type'].cat.codes
df['Is_Spouse']=df['Is_Spouse'].cat.codes
df['City_Code']=df['City_Code'].cat.codes

# df1['Health Indicator'].isnull().sum()

In [12]:
df2=df.copy()

In [13]:
my_imputer = SimpleImputer()
data_with_imputed_values = pd.DataFrame(my_imputer.fit_transform(df2))

In [14]:
data_with_imputed_values.columns=df2.columns
data_with_imputed_values.index=df2.index

# OUTLIER ANALYSIS

In [15]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [16]:
df2.shape

(50882, 14)

In [17]:
data_with_imputed_values=remove_outlier(data_with_imputed_values,'Reco_Policy_Premium')

In [18]:
data_with_imputed_values.shape

(50061, 14)

# standardization

In [19]:
data_with_imputed_values[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']]=StandardScaler().fit_transform(data_with_imputed_values[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']])

# Splitting of Dataset

In [20]:
d1=data_with_imputed_values[:500]
data_with_imputed_values=data_with_imputed_values[500:]

In [21]:
X=data_with_imputed_values.loc[:, data_with_imputed_values.columns != 'ID']
y=data_with_imputed_values['Response']
X.drop(columns=['Response'],inplace=True)

C:\Python\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
data_with_imputed_values.columns

Index(['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium', 'Response'],
      dtype='object')

In [23]:
#Best set of columns
X=X[['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age', 'Reco_Policy_Cat','Reco_Policy_Premium','Reco_Insurance_Type']]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.001, random_state=100)

In [26]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(y_train)
test_y = encoder.fit_transform(y_test)

In [27]:
smote = SMOTE(k_neighbors=5, n_jobs=-1,random_state=777)
smote_enn = make_pipeline(SimpleImputer(), SMOTEENN(smote=smote))
sm_xtrain, sm_ytrain = smote_enn.fit_resample(X, y)

# RANDOM FOREST

In [176]:
from sklearn.ensemble import RandomForestClassifier
rndmf = SelectFromModel(RandomForestClassifier(n_estimators=500,class_weight='balanced'))
rndmf.fit(sm_xtrain, sm_ytrain)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight='balanced',
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=500, n_jobs=None,
                                                 oob_score=

In [177]:
selected_feat= X_train.columns[(rndmf.get_support())]
print(selected_feat)

Index(['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age', 'Reco_Policy_Cat',
       'Reco_Policy_Premium'],
      dtype='object')


In [178]:
# Results from Grid Search 

#  The best estimator across ALL searched params:
#  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
#                        criterion='gini', max_depth=30, max_features='auto',
#                        max_leaf_nodes=None, max_samples=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=1, min_samples_split=2,
#                        min_weight_fraction_leaf=0.0, n_estimators=500,
#                        n_jobs=None, oob_score=False, random_state=None,
#                        verbose=0, warm_start=False)

#  The best score across ALL searched params:
#  0.837041945489787

#  The best parameters across ALL searched params:
#  {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}

In [28]:
RF_model= RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
RF_model=RF_model.fit(sm_xtrain, sm_ytrain)
y_pred1=RF_model.predict(X_test)

In [25]:
y_pred1_proba=RF_model.predict_proba(X_test)

In [30]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred1)

0.787941787941788

In [31]:
report = classification_report(test_y,y_pred1)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.73      0.82        37
           1       0.52      0.85      0.65        13

    accuracy                           0.76        50
   macro avg       0.73      0.79      0.73        50
weighted avg       0.83      0.76      0.77        50



In [32]:
Conf_metrics_sm_rndm = metrics.confusion_matrix(test_y, y_pred1,labels=[0,1])
print(Conf_metrics_sm_rndm)

[[27 10]
 [ 2 11]]


# LIGHT GBM

In [86]:
from lightgbm import LGBMClassifier
LightGBM=LGBMClassifier(random_state=2)
LightGBM.fit(sm_xtrain,sm_ytrain)
y_pred5=LightGBM.predict(X_test)

In [87]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred5)

0.632016632016632

In [32]:
# fit_params={"early_stopping_rounds":30, 
#             "eval_metric" : 'auc', 
#             "eval_set" : [(X_test,y_test)],
#             'eval_names': ['valid'],
#             #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
#             'verbose': 100,
#             'categorical_feature': 'auto'}

In [29]:
# from scipy.stats import randint as sp_randint
# from scipy.stats import uniform as sp_uniform
# param_test ={'num_leaves': sp_randint(6, 50), 
#              'min_child_samples': sp_randint(100, 500), 
#              'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
#              'subsample': sp_uniform(loc=0.2, scale=0.8), 
#              'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
#              'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
#              'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [30]:
# #This parameter defines the number of HP points to be tested
# n_HP_points_to_test = 100

# import lightgbm as lgb
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# #n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
# clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
# gs = RandomizedSearchCV(
#     estimator=clf, param_distributions=param_test, 
#     n_iter=n_HP_points_to_test,
#     scoring='roc_auc',
#     cv=3,
#     refit=True,
#     random_state=314,
#     verbose=True)


In [35]:
opt_parameters= {'colsample_bytree': 0.6847074798584072, 'min_child_samples': 125, 'min_child_weight': 10.0, 'num_leaves': 49, 'reg_alpha': 5, 'reg_lambda': 50, 'subsample': 0.20892704149103666}

In [38]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
clf_sw = lgb.LGBMClassifier(**clf.get_params())
#set optimal parameters
clf_sw.set_params(**opt_parameters)

LGBMClassifier(boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.6847074798584072, importance_type='split',
               learning_rate=0.1, max_depth=-1, metric='None',
               min_child_samples=125, min_child_weight=10.0, min_split_gain=0.0,
               n_estimators=5000, n_jobs=4, num_leaves=49, objective=None,
               random_state=314, reg_alpha=5, reg_lambda=50, silent=True,
               subsample=0.20892704149103666, subsample_for_bin=200000,
               subsample_freq=0)

In [39]:
gs=clf_sw.fit(sm_xtrain, sm_ytrain)

In [40]:
y_pred6 = gs.predict(X_test)

In [33]:
# gs.fit(sm_xtrain, sm_ytrain, **fit_params)
# print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[2]	valid's auc: 0.637214
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[9]	valid's auc: 0.674636


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.682952
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[2]	valid's auc: 0.672557
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.7079
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[3]	valid's auc: 0.691268
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[11]	valid's auc: 0.694387
Training until validation scores don't i

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   37.9s finished


[100]	valid's auc: 0.696466
[200]	valid's auc: 0.756757
Early stopping, best iteration is:
[188]	valid's auc: 0.760915
Best score reached: 0.8930842127162499 with params: {'colsample_bytree': 0.6847074798584072, 'min_child_samples': 125, 'min_child_weight': 10.0, 'num_leaves': 49, 'reg_alpha': 5, 'reg_lambda': 50, 'subsample': 0.20892704149103666} 


In [41]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred6)

0.7785862785862786

In [42]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred5)

0.632016632016632

# Decison tree

In [43]:
from sklearn.tree import DecisionTreeClassifier
dt_cls = DecisionTreeClassifier(random_state=101)
dt_cls.fit(sm_xtrain,sm_ytrain)
y_pred2=dt_cls.predict(X_test)                                           

In [44]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred2)

0.9074844074844075

In [45]:
report = classification_report(test_y,y_pred2)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.89      0.93        37
           1       0.75      0.92      0.83        13

    accuracy                           0.90        50
   macro avg       0.86      0.91      0.88        50
weighted avg       0.91      0.90      0.90        50



In [35]:
Conf_metrics_sm_rndm = metrics.confusion_matrix(test_y, y_pred2,labels=[0,1])
print(Conf_metrics_sm_rndm)

[[30  7]
 [ 2 11]]


# CAT BOOST

In [46]:
CatBoost=CatBoostClassifier(random_state=101)                                                                                                   
CatBoost.fit(sm_xtrain,sm_ytrain)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 66.4ms	remaining: 1m 6s
1:	learn: 0.6679542	total: 76.5ms	remaining: 38.2s
2:	learn: 0.6586634	total: 85.4ms	remaining: 28.4s
3:	learn: 0.6506448	total: 95.1ms	remaining: 23.7s
4:	learn: 0.6408058	total: 106ms	remaining: 21.2s
5:	learn: 0.6340672	total: 118ms	remaining: 19.5s
6:	learn: 0.6286264	total: 131ms	remaining: 18.5s
7:	learn: 0.6226329	total: 143ms	remaining: 17.7s
8:	learn: 0.6184221	total: 155ms	remaining: 17s
9:	learn: 0.6140516	total: 166ms	remaining: 16.4s
10:	learn: 0.6095748	total: 178ms	remaining: 16s
11:	learn: 0.6056595	total: 197ms	remaining: 16.2s
12:	learn: 0.6009331	total: 212ms	remaining: 16.1s
13:	learn: 0.5953545	total: 224ms	remaining: 15.8s
14:	learn: 0.5916079	total: 238ms	remaining: 15.6s
15:	learn: 0.5871947	total: 251ms	remaining: 15.4s
16:	learn: 0.5799814	total: 264ms	remaining: 15.3s
17:	learn: 0.5771092	total: 277ms	remaining: 15.1s
18:	learn: 0.5719313	total: 291ms	remaining: 15s
19:	learn: 0.5

In [47]:
y_pred3=CatBoost.predict(np.array(X_test))

In [48]:
#reco insurance type
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y,  y_pred3)

0.6746361746361745

In [273]:
Conf_metrics_sm_cat = metrics.confusion_matrix(test_y, y_pred3,labels=[0,1])
print(Conf_metrics_sm_cat)

[[30  7]
 [ 7  6]]


# ENSEMBLE

In [51]:
from sklearn.ensemble import VotingClassifier
model6 = VotingClassifier(estimators=[('Cb', CatBoost),('dt', dt_cls),('rf', RF_model)], voting='soft')
model6.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred_6=model6.predict(X_test)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 28ms	remaining: 28s
1:	learn: 0.6679542	total: 49.7ms	remaining: 24.8s
2:	learn: 0.6586634	total: 65.8ms	remaining: 21.9s
3:	learn: 0.6506448	total: 82.9ms	remaining: 20.6s
4:	learn: 0.6408058	total: 96.4ms	remaining: 19.2s
5:	learn: 0.6340672	total: 111ms	remaining: 18.3s
6:	learn: 0.6286264	total: 125ms	remaining: 17.8s
7:	learn: 0.6226329	total: 140ms	remaining: 17.4s
8:	learn: 0.6184221	total: 153ms	remaining: 16.9s
9:	learn: 0.6140516	total: 169ms	remaining: 16.7s
10:	learn: 0.6095748	total: 182ms	remaining: 16.4s
11:	learn: 0.6056595	total: 196ms	remaining: 16.2s
12:	learn: 0.6009331	total: 210ms	remaining: 15.9s
13:	learn: 0.5953545	total: 223ms	remaining: 15.7s
14:	learn: 0.5916079	total: 236ms	remaining: 15.5s
15:	learn: 0.5871947	total: 250ms	remaining: 15.4s
16:	learn: 0.5799814	total: 265ms	remaining: 15.3s
17:	learn: 0.5771092	total: 280ms	remaining: 15.3s
18:	learn: 0.5719313	total: 297ms	remaining: 15.3s
19:	learn: 

In [88]:
from sklearn.ensemble import VotingClassifier
model5 = VotingClassifier(estimators=[('lgbm', gs),('dt', dt_cls),('rf', RF_model)], voting='soft')
model5.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred_5=model5.predict(X_test)

Model Complete


In [53]:
from sklearn.ensemble import VotingClassifier
model4 = VotingClassifier(estimators=[('Cb', CatBoost),('lgbm', gs),('dt', dt_cls),('rf', RF_model)], voting='soft')
model4.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred_4=model4.predict(X_test)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 15.8ms	remaining: 15.8s
1:	learn: 0.6679542	total: 32.3ms	remaining: 16.1s
2:	learn: 0.6586634	total: 44.4ms	remaining: 14.8s
3:	learn: 0.6506448	total: 58.4ms	remaining: 14.5s
4:	learn: 0.6408058	total: 76.1ms	remaining: 15.1s
5:	learn: 0.6340672	total: 98ms	remaining: 16.2s
6:	learn: 0.6286264	total: 114ms	remaining: 16.2s
7:	learn: 0.6226329	total: 136ms	remaining: 16.9s
8:	learn: 0.6184221	total: 151ms	remaining: 16.6s
9:	learn: 0.6140516	total: 167ms	remaining: 16.5s
10:	learn: 0.6095748	total: 183ms	remaining: 16.5s
11:	learn: 0.6056595	total: 203ms	remaining: 16.7s
12:	learn: 0.6009331	total: 218ms	remaining: 16.6s
13:	learn: 0.5953545	total: 235ms	remaining: 16.6s
14:	learn: 0.5916079	total: 251ms	remaining: 16.5s
15:	learn: 0.5871947	total: 275ms	remaining: 16.9s
16:	learn: 0.5799814	total: 292ms	remaining: 16.9s
17:	learn: 0.5771092	total: 311ms	remaining: 17s
18:	learn: 0.5719313	total: 330ms	remaining: 17s
19:	learn: 0

In [54]:
from sklearn.ensemble import VotingClassifier
model3 = VotingClassifier(estimators=[('Cb', CatBoost),('lgbm', gs),('dt', dt_cls)], voting='soft')
model3.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred_3=model3.predict(X_test)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 11ms	remaining: 11s
1:	learn: 0.6679542	total: 23.3ms	remaining: 11.6s
2:	learn: 0.6586634	total: 34.7ms	remaining: 11.5s
3:	learn: 0.6506448	total: 47.8ms	remaining: 11.9s
4:	learn: 0.6408058	total: 66.2ms	remaining: 13.2s
5:	learn: 0.6340672	total: 81.2ms	remaining: 13.4s
6:	learn: 0.6286264	total: 100ms	remaining: 14.2s
7:	learn: 0.6226329	total: 114ms	remaining: 14.2s
8:	learn: 0.6184221	total: 138ms	remaining: 15.1s
9:	learn: 0.6140516	total: 153ms	remaining: 15.2s
10:	learn: 0.6095748	total: 177ms	remaining: 15.9s
11:	learn: 0.6056595	total: 196ms	remaining: 16.2s
12:	learn: 0.6009331	total: 218ms	remaining: 16.6s
13:	learn: 0.5953545	total: 234ms	remaining: 16.5s
14:	learn: 0.5916079	total: 258ms	remaining: 16.9s
15:	learn: 0.5871947	total: 275ms	remaining: 16.9s
16:	learn: 0.5799814	total: 302ms	remaining: 17.5s
17:	learn: 0.5771092	total: 318ms	remaining: 17.3s
18:	learn: 0.5719313	total: 336ms	remaining: 17.4s
19:	learn:

In [55]:
from sklearn.ensemble import VotingClassifier
model2 = VotingClassifier(estimators=[('Cb', CatBoost),('lgbm', gs)], voting='soft')
model2.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred_2=model2.predict(X_test)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 12.7ms	remaining: 12.7s
1:	learn: 0.6679542	total: 22.9ms	remaining: 11.4s
2:	learn: 0.6586634	total: 34.3ms	remaining: 11.4s
3:	learn: 0.6506448	total: 48.7ms	remaining: 12.1s
4:	learn: 0.6408058	total: 67ms	remaining: 13.3s
5:	learn: 0.6340672	total: 82.7ms	remaining: 13.7s
6:	learn: 0.6286264	total: 98ms	remaining: 13.9s
7:	learn: 0.6226329	total: 117ms	remaining: 14.6s
8:	learn: 0.6184221	total: 133ms	remaining: 14.6s
9:	learn: 0.6140516	total: 146ms	remaining: 14.5s
10:	learn: 0.6095748	total: 161ms	remaining: 14.4s
11:	learn: 0.6056595	total: 178ms	remaining: 14.6s
12:	learn: 0.6009331	total: 197ms	remaining: 14.9s
13:	learn: 0.5953545	total: 222ms	remaining: 15.7s
14:	learn: 0.5916079	total: 239ms	remaining: 15.7s
15:	learn: 0.5871947	total: 257ms	remaining: 15.8s
16:	learn: 0.5799814	total: 273ms	remaining: 15.8s
17:	learn: 0.5771092	total: 290ms	remaining: 15.8s
18:	learn: 0.5719313	total: 306ms	remaining: 15.8s
19:	learn

In [56]:
from sklearn.ensemble import VotingClassifier
model1 = VotingClassifier(estimators=[('Cb', CatBoost),('rf', RF_model),('lgbm', gs)], voting='soft')
model1.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred_1=model1.predict(X_test)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 17.2ms	remaining: 17.2s
1:	learn: 0.6679542	total: 34.5ms	remaining: 17.2s
2:	learn: 0.6586634	total: 60.9ms	remaining: 20.2s
3:	learn: 0.6506448	total: 78.6ms	remaining: 19.6s
4:	learn: 0.6408058	total: 105ms	remaining: 20.9s
5:	learn: 0.6340672	total: 123ms	remaining: 20.4s
6:	learn: 0.6286264	total: 157ms	remaining: 22.2s
7:	learn: 0.6226329	total: 177ms	remaining: 21.9s
8:	learn: 0.6184221	total: 194ms	remaining: 21.4s
9:	learn: 0.6140516	total: 209ms	remaining: 20.7s
10:	learn: 0.6095748	total: 228ms	remaining: 20.5s
11:	learn: 0.6056595	total: 247ms	remaining: 20.4s
12:	learn: 0.6009331	total: 269ms	remaining: 20.4s
13:	learn: 0.5953545	total: 295ms	remaining: 20.8s
14:	learn: 0.5916079	total: 314ms	remaining: 20.6s
15:	learn: 0.5871947	total: 332ms	remaining: 20.4s
16:	learn: 0.5799814	total: 365ms	remaining: 21.1s
17:	learn: 0.5771092	total: 383ms	remaining: 20.9s
18:	learn: 0.5719313	total: 405ms	remaining: 20.9s
19:	lear

In [57]:
from sklearn.ensemble import VotingClassifier
model = VotingClassifier(estimators=[('Cb', CatBoost),('rf', RF_model),('lgbm', LightGBM)], voting='soft')
model.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred=model.predict(X_test)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 16.5ms	remaining: 16.5s
1:	learn: 0.6679542	total: 35.6ms	remaining: 17.8s
2:	learn: 0.6586634	total: 49ms	remaining: 16.3s
3:	learn: 0.6506448	total: 76.3ms	remaining: 19s
4:	learn: 0.6408058	total: 111ms	remaining: 22.2s
5:	learn: 0.6340672	total: 129ms	remaining: 21.4s
6:	learn: 0.6286264	total: 153ms	remaining: 21.6s
7:	learn: 0.6226329	total: 168ms	remaining: 20.9s
8:	learn: 0.6184221	total: 184ms	remaining: 20.3s
9:	learn: 0.6140516	total: 202ms	remaining: 20s
10:	learn: 0.6095748	total: 218ms	remaining: 19.6s
11:	learn: 0.6056595	total: 236ms	remaining: 19.5s
12:	learn: 0.6009331	total: 254ms	remaining: 19.3s
13:	learn: 0.5953545	total: 274ms	remaining: 19.3s
14:	learn: 0.5916079	total: 292ms	remaining: 19.2s
15:	learn: 0.5871947	total: 310ms	remaining: 19s
16:	learn: 0.5799814	total: 326ms	remaining: 18.8s
17:	learn: 0.5771092	total: 343ms	remaining: 18.7s
18:	learn: 0.5719313	total: 366ms	remaining: 18.9s
19:	learn: 0.568

In [47]:
Conf_metrics_emsem = metrics.confusion_matrix(test_y, Ensemble_pred,labels=[0,1])
print(Conf_metrics_emsem)

[[30  7]
 [ 9  4]]


In [276]:
Conf_metrics_emsem = metrics.confusion_matrix(test_y, Ensemble_pred,labels=[0,1])
print(Conf_metrics_emsem)

[[30  7]
 [ 9  4]]


# VALIDATION

In [89]:
X1=d1.loc[:, d1.columns != 'ID']
y1=d1['Response']
X1.drop(columns=['Response'],inplace=True)
X1=X1[['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age','Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat','Reco_Policy_Premium','Holding_Policy_Type','Reco_Insurance_Type','Is_Spouse']]

C:\Python\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [90]:
X.columns

Index(['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age', 'Reco_Policy_Cat',
       'Reco_Policy_Premium', 'Reco_Insurance_Type'],
      dtype='object')

In [60]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X,y, test_size=0.20, random_state=100)

In [62]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.652 (0.009)


In [63]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model1
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.647 (0.008)


In [64]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model2
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.650 (0.008)


In [65]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model3
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.642 (0.010)


In [66]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model4
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.641 (0.010)


In [67]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model5
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.636 (0.010)


In [68]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model6
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.639 (0.010)


# TEST

In [91]:
Test=pd.read_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\test_YCcRUnU.csv')

In [92]:
Test.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,C1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,11934.0
1,50884,C4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,32204.8
2,50885,C1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9240.0
3,50886,C3,1177,Rented,Individual,23,23,No,X3,3.0,3.0,18,9086.0
4,50887,C1,951,Owned,Individual,75,75,No,X3,NaN,NaN,5,22534.0


# DATA Pre_Processing

In [93]:
Test['Holding_Policy_Duration']=Test['Holding_Policy_Duration'].replace('14+',15)

In [94]:
Test=Test.astype({"Holding_Policy_Type":'category', "Reco_Policy_Cat":'category',"City_Code":'category',"Accomodation_Type":'category',"Reco_Insurance_Type":'category',"Holding_Policy_Duration":'category',"Is_Spouse":'category',"Health Indicator":'category','Region_Code':'category'})

In [95]:
Test['Accomodation_Type']=Test['Accomodation_Type'].cat.codes
Test['Reco_Insurance_Type']=Test['Reco_Insurance_Type'].cat.codes
Test['City_Code']=Test['City_Code'].cat.codes

In [96]:
Test['Health Indicator']=Test['Health Indicator'].cat.codes
Test['Health Indicator']=Test['Health Indicator'].replace(-1, np.nan)

In [97]:
Test.columns

Index(['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium'],
      dtype='object')

In [98]:
Test=Test[['ID','City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age','Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat','Reco_Policy_Premium','Holding_Policy_Type','Accomodation_Type','Reco_Insurance_Type']]

In [99]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
Test_data = pd.DataFrame(my_imputer.fit_transform(Test))
Test_data.columns=Test.columns
Test_data.index=Test.index

In [100]:
Test_data[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']]=StandardScaler().fit_transform(Test_data[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']])

In [101]:
Final_Test_data=Test_data.drop(columns=['ID'])

In [102]:
Final_Test_data=Final_Test_data[['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age', 'Reco_Policy_Cat','Reco_Policy_Premium','Reco_Insurance_Type']]

In [103]:
Final_Test_data.head()

,City_Code,Region_Code,Upper_Age,Lower_Age,Reco_Policy_Cat,Reco_Policy_Premium,Reco_Insurance_Type
0,-1.231415,-1.107355,-0.862252,-0.738218,-1.608585,-0.351856,0.0
1,1.249651,-1.210948,1.398027,1.462293,0.454036,2.767762,1.0
2,-1.231415,-0.823692,-0.978164,-0.854035,0.295373,-0.766455,0.0
3,0.588033,-0.397502,-1.267943,-1.143576,0.454036,-0.790155,0.0
4,-1.231415,-0.554629,1.745763,1.867650,-1.608585,1.279454,0.0


# ModelTesting

In [104]:
y_final=model.predict_proba(Final_Test_data)[:,1]

In [105]:
y_final_1=model1.predict_proba(Final_Test_data)[:,1]

In [106]:
y_final_2=model2.predict_proba(Final_Test_data)[:,1]

In [107]:
y_final_3=model3.predict_proba(Final_Test_data)[:,1]

In [108]:
y_final_4=model4.predict_proba(Final_Test_data)[:,1]

In [109]:
y_final_5=model5.predict_proba(Final_Test_data)[:,1]

In [110]:
y_final_6=model6.predict_proba(Final_Test_data)[:,1]

In [126]:
LightGBM.predict_proba(Final_Test_data)[:,1]

array([0.96803705, 0.9788022 , 0.99066628, ..., 0.00273949, 0.99466306,
       0.96389664])

In [146]:
Test_data['Response']=y_final_6

In [148]:
y_final_6

array([0.87813971, 0.84306   , 0.98260661, ..., 0.01644177, 0.97921859,
       0.91489735])

In [149]:
result=Test_data[['ID','Response']]

In [150]:
result['Response'].value_counts()

0.879124    2
0.070788    2
0.830233    2
0.874486    2
0.962746    2
           ..
0.521848    1
0.990588    1
0.921108    1
0.970268    1
0.899976    1
Name: Response, Length: 21791, dtype: int64

In [ ]:
result.head()

In [151]:
result.to_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\model_6.csv')